In [ ]:
# Libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.polynomial.hermite import hermgauss
from scipy import integrate, stats, special, optimize

from scipy.stats import norm
from scipy.integrate import quad, dblquad
from scipy.special import logsumexp, expit
from scipy.optimize import minimize

# Part 2: Numerical Integration

In [ ]:
np.random.seed(4309)
X = 0.5

##### 1: Creating function #####
def binomial_logit(beta_i, pdf = norm.pdf, mu = 0, sigma = 1):
  logit_prob = expit(beta_i * X)
  density = pdf(beta_i, loc = mu, scale = sigma)
  return (logit_prob * density)



##### 2: True integral #####
true_int, _ = quad(binomial_logit,
                   -np.inf,
                   np.inf,
                   args = (norm.pdf, 0.5, np.sqrt(2)),
                   epsabs = 1e-14)



##### 3: Monte-Carlo integral #####
draw_1 = np.random.normal(loc = 0.5,
                          scale = np.sqrt(2),
                          size = 20)

draw_2 = np.random.normal(loc = 0.5,
                          scale = np.sqrt(2),
                          size = 400)

int_1 = np.mean(expit(draw_1 * X))
int_2 = np.mean(expit(draw_2 * X))



###### 4: Gauss-Hermite integral #####
# For k = 4
nodes4, weights4 = hermgauss(4)
int_gh4 = (1/np.sqrt(np.pi)) * np.sum(weights4 * expit(((np.sqrt(2) * np.sqrt(2) * nodes4) + 0.5)))

# For k = 12
nodes12, weights12 = hermgauss(12)
int_gh12 = (1/np.sqrt(np.pi)) * np.sum(weights12 * expit(((np.sqrt(2) * np.sqrt(2) * nodes12) + 0.5)))

# For k = 15
nodes15, weights15 = hermgauss(15)
int_gh15 = (1/np.sqrt(np.pi)) * np.sum(weights15 * expit(((np.sqrt(2) * np.sqrt(2) * nodes15) + 0.5)))



##### 6: Same procedure for 2D case #####
X1, X2 = 0.5, 1
mu1, mu2 = 0.5, 1
sigma1, sigma2 = 2, 1

##### Creating function #####
def binomial_logit_2d(beta1, beta2,
                      pdf = norm.pdf,
                      mu1 = mu1, sigma1 = sigma1,
                      mu2 = mu2, sigma2 = sigma2,
                      X1 = X1, X2 = X2):
    """
    Returns the integrand for the 2D binomial logit probability:
    expit(0.5*beta1 + 1*beta2) times the product of two independent normal densities.
    """
    util = X1 * beta1 + X2 * beta2
    return expit(util) * pdf(beta1, loc = mu1, scale = sigma1) * pdf(beta2, loc = mu2, scale = sigma2)

##### True integral #####
true_int_2d, err_2d = dblquad(
                              lambda b2, b1: binomial_logit_2d(b1, b2),
                              -np.inf, np.inf,
                              lambda b1: -np.inf,
                              lambda b1: np.inf,
                              epsabs=1e-14
                             )

##### Monte-Carlo integral #####
draws_20_b1 = np.random.normal(loc = mu1, scale = sigma1, size = 20)
draws_20_b2 = np.random.normal(loc = mu2, scale = sigma2, size = 20)
mc_int_20 = np.mean(expit(X1*draws_20_b1 + X2*draws_20_b2))

draws_400_b1 = np.random.normal(loc=mu1, scale=sigma1, size=400)
draws_400_b2 = np.random.normal(loc=mu2, scale=sigma2, size=400)
mc_int_400 = np.mean(expit(X1*draws_400_b1 + X2*draws_400_b2))

###### Gauss-Hermite integral #####
k = 5
nodes, weights = hermgauss(k)

# In 1-D we use: beta = sqrt(2)*sigma*x + mu, and then the approximation is
# I ≈ 1/√π Σ w_i * expit( (√2*σ*x_i + mu)*X )
# In 2-D, the joint transformation gives a prefactor 1/π.
gh_sum = 0.0
for i in range(k):
    for j in range(k):
        beta1 = np.sqrt(2)*sigma1*nodes[i] + mu1
        beta2 = np.sqrt(2)*sigma2*nodes[j] + mu2
        gh_sum += weights[i] * weights[j] * expit(X1*beta1 + X2*beta2)

gh_int_2d = gh_sum / np.pi  # because (1/√π)² = 1/π



##### 7: Comparison tables #####
# Calculating errors for each method
error_int_1 = abs(int_1 - true_int)
error_int_2 = abs(int_2 - true_int)
error_int_gh4 = abs(int_gh4 - true_int)
error_int_gh12 = abs(int_gh12 - true_int)
error_int_gh15 = abs(int_gh15 - true_int)

error_mc_int_20 = abs(mc_int_20 - true_int_2d)
error_mc_int_400 = abs(mc_int_400 - true_int_2d)
error_gh_int_2d = abs(gh_int_2d - true_int_2d)

# Creating 1D table
data_1d = {
    "Method": ["True integral", "Monte Carlo (20)", "Monte Carlo (400)",
               "Gauss-Hermite (4)", "Gauss-Hermite (12)", "Gauss-Hermite (15)"],
    "Approximation value": [true_int, int_1, int_2, int_gh4, int_gh12, int_gh15],
    "True value": [true_int] * 6,
    "Error": [0, error_int_1, error_int_2, error_int_gh4, error_int_gh12, error_int_gh15],
    "Number of points used": [np.nan, 20, 400, 4, 12, 15]
}

df_1d = pd.DataFrame(data_1d)

# Creating 2D table
data_2d = {
    "Method": ["True integral", "Monte Carlo (20)", "Monte Carlo (400)",
               "Gauss-Hermite (5)"],
    "Approximation value": [true_int_2d, mc_int_20, mc_int_400, gh_int_2d],
    "True value": [true_int_2d] * 4,
    "Error": [0, error_mc_int_20, error_mc_int_400, error_gh_int_2d],
    "Number of points used": [np.nan, 20*2, 400*2, 25]  # 20 pairs = 40, 400 pairs = 800, 5x5 for 2D Gauss-Hermite
}

df_2d = pd.DataFrame(data_2d)

In [ ]:
df_1d

In [ ]:
df_2d

# Part 3: Maximum Likelihood

In [ ]:
data = pd.read_csv('pset1_data.csv').drop(columns = 'Unnamed: 0')

# Input data on outside option explicitly
outside_option = (
                  data.groupby('individual_id_i')
                  .apply(lambda x: pd.Series({
                      'apt_id_j': 20, #index for outside option
                      'logsqfeet_j': 0,
                      'bath_j': 0,
                      'outdoor_j': 0,
                      'y_ij': 1 if x['y_ij'].sum() == 0 else 0,
                      'family_size_i': x['family_size_i'].iloc[0]
                  }, index = ['apt_id_j', 'logsqfeet_j',
                              'bath_j', 'outdoor_j',
                              'y_ij', 'family_size_i']))
                  .reset_index()
                 )

# Merge datasets
full_data = pd.concat([
                        data[['individual_id_i', 'apt_id_j',
                              'family_size_i', 'logsqfeet_j',
                              'bath_j', 'outdoor_j', 'y_ij']],
                        outside_option
                       ],
                       ignore_index = True)

# Create feature associated with β3 and the intercept
full_data['bath_family'] = full_data['bath_j'] * full_data['family_size_i']
full_data['intercept'] = 1

# Verify exactly 1 choice per individual
assert (full_data.groupby('individual_id_i')['y_ij'].sum() == 1).all()



##### 1: Plain logit #####
# Computing the negative log likelihood using the max trick in Part 1
def neg_log_lik(params):

    # Define predictors used in the utility function
    feature_columns = ['intercept', 'logsqfeet_j',
                       'bath_j', 'bath_family', 'outdoor_j']

    # Compute utilities for all alternatives
    utilities = full_data[feature_columns].dot(params)

    # Compute per-individual maximum utility (for numerical stability)
    group_max = utilities.groupby(full_data['individual_id_i']).transform('max')

    # Compute exponentiated utilities relative to the group max
    exp_term = np.exp(utilities - group_max)

    # Compute the sum of exponentiated utilities for each individual
    sum_exp = exp_term.groupby(full_data['individual_id_i']).transform('sum')

    # Compute log-sum-exp for each observation (each group has the same value)
    logsumexp_series = group_max + np.log(sum_exp)

    # Identify the index of the chosen alternative per individual using the indicator variable
    chosen_idx = full_data.groupby('individual_id_i')['y_ij'].idxmax()

    # Extract the chosen utility and corresponding log-sum-exp for each individual
    chosen_utilities = utilities.loc[chosen_idx]
    group_logsumexp = logsumexp_series.loc[chosen_idx]

    # Sum over individuals and return the negative log-likelihood for minimization
    return -(chosen_utilities - group_logsumexp).sum()

# Optimization
result = minimize(
                  neg_log_lik,
                  x0 = np.zeros(5),
                  method = 'BFGS',
                  jac = False,
                  hess = True,
                  options = {
                      'disp': False,
                      'maxiter': 1000,
                      'gtol': 1e-6,
                      'ftol': 1e-6,
                  }
                  )



##### 2: Mixed logit via Gauss-Hermite
# Pre-calculate chosen indices per individual from full_data:
chosen_idx = full_data.groupby('individual_id_i')['y_ij'].idxmax()
groups = full_data['individual_id_i']

def neg_log_lik_mixed(params, gh_nodes, gh_weights):
    beta0, beta1, beta2, beta3, gamma, sigma = params
    X = full_data[['intercept', 'logsqfeet_j', 'bath_j', 'bath_family']].values
    outdoor = full_data['outdoor_j'].values

    # Pre-calculate the fixed (non-random) component
    fixed_util = X.dot(np.array([beta0, beta1, beta2, beta3]))

    unique_ids = full_data['individual_id_i'].unique()
    likelihood = pd.Series(0.0, index = unique_ids)

    # Loop over the Gauss-Hermite nodes and weights
    for x, w in zip(gh_nodes, gh_weights):
        adjust = gamma + sigma * np.sqrt(2) * x
        util = fixed_util + adjust * outdoor

        # For numerical stability compute group maximum utility per individual:
        util_series = pd.Series(util, index = full_data.index)
        group_max = util_series.groupby(groups).transform('max')
        exp_util = np.exp(util - group_max)
        # Sum exp utilities by individual (denom of logit probability)
        denom = exp_util.groupby(groups).transform('sum')

        # For each individual, get the probability of the chosen alternative.
        p_chosen = np.exp(util_series.loc[chosen_idx] - group_max.loc[chosen_idx]) / denom.loc[chosen_idx]

        likelihood += (w / np.sqrt(np.pi)) * p_chosen

    # Avoid numerical issues by bounding from below
    likelihood = likelihood.clip(lower = 1e-300)
    # Negative log likelihood: sum over individuals
    nll = -np.log(likelihood).sum()
    return nll

# Set up Gauss-Hermite quadrature with a chosen number of nodes (say, 20 nodes)
n_nodes = 5
gh_nodes, gh_weights = np.polynomial.hermite.hermgauss(n_nodes)

result_mixed = minimize(
                        neg_log_lik_mixed,
                        x0 = np.zeros(6),
                        args = (gh_nodes, gh_weights),
                        method = 'BFGS',
                        jac = False,
                        hess = True,
                        options = {
                                'disp': False,
                                'maxiter': 1000,
                                'gtol': 1e-6,
                                'ftol': 1e-6,
                        }
                        )



##### 3: Standard errors from mixed logit #####
# Calculate the Hessian matrix (information matrix)
hessian_plain = result.hess_inv

# Compute the standard errors (square roots of diagonal elements of the covariance matrix)
std_errors_plain = np.sqrt(np.diag(hessian_plain))

# Calculate the Hessian matrix (information matrix)
hessian_mixed = result_mixed.hess_inv

# Compute the standard errors (square roots of diagonal elements of the covariance matrix)
std_errors_mixed = np.sqrt(np.diag(hessian_mixed))

# Report results:
print("\nPlain logit")
print(f"Final Estimates:\n{pd.Series(result.x, index = ['β0', 'β1','β2', 'β3', 'γ'])}")
print(f"\nStandard errors for plain logit estimates:\n{pd.Series(std_errors_plain, index = ['β0', 'β1', 'β2', 'β3', 'γ'])}")

print("\nMixed Logit via Gauss-Hermite Integration")
print(f"Final Estimates:\n{pd.Series(result_mixed.x, index = ['β0', 'β1', 'β2', 'β3', 'γ', 'σ'])}")
print(f"\nStandard errors for Gauss Hermite estimates:\n{pd.Series(std_errors_mixed, index = ['β0', 'β1', 'β2', 'β3', 'γ', 'σ'])}")